In [3]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import neighbors, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
movies_data = pd.read_csv("movies.csv")
ratings_data = pd.read_csv("ratings.csv")

In [5]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703.0
1,1,3,4.0,964981247.0
2,1,6,4.0,964982224.0
3,1,47,5.0,964983815.0
4,1,50,5.0,964982931.0


In [7]:
ratings_data.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,43084.0,1.452162e+02,8.559502e+01,1.0,68.0,1.400000e+02,2.210000e+02,2.880000e+02
movieId,43084.0,1.858233e+04,3.528132e+04,1.0,1090.0,2.791000e+03,7.155000e+03,1.935870e+05
rating,43084.0,3.566951e+00,1.022535e+00,0.5,3.0,4.000000e+00,4.000000e+00,5.000000e+00
timestamp,43083.0,1.181536e+09,2.146729e+08,829322340.0,978559632.5,1.161566e+09,1.393542e+09,1.537650e+09


In [8]:
movies_data.describe().T

,count,mean,std,min,25%,50%,75%,max
movieId,9742.0,42200.353623,52160.494854,1.0,3248.25,7300.0,76232.0,193609.0


In [9]:
movies_rating_df = ratings_data.merge(movies_data, how="left", on="movieId")

In [10]:
movies_rating_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [11]:
mean_rating = pd.DataFrame(movies_rating_df.groupby("title")["rating"].mean())

In [12]:
mean_rating.head(10)

,rating
title,
'Til There Was You (1997),3.000000
'Tis the Season for Love (2015),1.500000
"'burbs, The (1989)",3.285714
(500) Days of Summer (2009),3.727273
00 Schneider - Jagd auf Nihil Baxter (1994),4.500000
1-900 (06) (1994),4.000000
10 (1979),2.750000
10 Cent Pistol (2015),0.500000
10 Cloverfield Lane (2016),3.928571


In [13]:
mean_rating["Count"] = movies_rating_df.groupby("title")["rating"].count()

In [14]:
movie_user = movies_rating_df.pivot_table(index="userId", columns="title", values="rating")

In [15]:
movie_user.head(10)

title,'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",(500) Days of Summer (2009),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),"10,000 BC (2008)",100 Girls (2000),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),10th & Wolf (2006),11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),127 Hours (2010),13 Assassins (Jûsan-nin no shikaku) (2010),13 Ghosts (1960),13 Going on 30 (2004),13 Hours (2016),13 Tzameti (2005),13th (2016),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1776 (1972),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),...,You Got Served (2004),You Only Live Twice (1967),You've Got Mail (1998),"You, Me and Dupree (2006)",Young Adult (2011),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Your Highness (2011),Your Name. (2016),"Yours, Mine and Ours (2005)",Youth (2015),Youth in Revolt (2009),Zabriskie Point (1970),Zack and Miri Make a Porno (2008),Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zootopia (2016),Zulu (1964),[REC] (2007),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [16]:
corr = movie_user.corrwith(movie_user["Aladdin (1992)"])

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [17]:
corr.head(10)

title
'Til There Was You (1997)                           NaN
'Tis the Season for Love (2015)                     NaN
'burbs, The (1989)                             0.866025
(500) Days of Summer (2009)                    0.377964
00 Schneider - Jagd auf Nihil Baxter (1994)         NaN
1-900 (06) (1994)                                   NaN
10 (1979)                                           NaN
10 Cent Pistol (2015)                               NaN
10 Cloverfield Lane (2016)                     0.500000
10 Items or Less (2006)                             NaN
dtype: float64

In [18]:
recommendation = pd.DataFrame(corr,columns=['Correlation'])
recommendation.dropna(inplace=True)
recommendation = recommendation.join(mean_rating['Count'])
recommendation.head()

,Correlation,Count
title,,
"'burbs, The (1989)",0.866025,7
(500) Days of Summer (2009),0.377964,22
10 Cloverfield Lane (2016),0.500000,7
10 Things I Hate About You (1999),-0.099388,21
"10,000 BC (2008)",0.406250,8


In [19]:
recc = recommendation[recommendation['Count']>100].sort_values('Correlation',ascending=False).reset_index()

In [20]:
recc.head(10)

,title,Correlation,Count
0,Toy Story (1995),0.697767,107
1,Forrest Gump (1994),0.653610,153
2,Jurassic Park (1993),0.382886,109
3,Terminator 2: Judgment Day (1991),0.266122,104
4,"Silence of the Lambs, The (1991)",0.257543,131
5,"Shawshank Redemption, The (1994)",0.239177,147
6,Star Wars: Episode V - The Empire Strikes Back...,0.194425,105
7,Pulp Fiction (1994),0.189453,151
8,Star Wars: Episode VI - Return of the Jedi (1983),0.180810,101
9,Braveheart (1995),0.140591,110
